In [1]:
import pandas as pd
import numpy as np
import gensim
from helpers import lemmatize_stemming, preprocess, compute_coherence_values

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline


filenames = !ls data/unzipped
filenames = [ 'data/unzipped/'+filename for filename in filenames ]

import random
indeces = random.sample(range(len(filenames)), 50)
# indeces = [68]

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
loaded_lda = gensim.models.LdaMulticore.load('trained_lda')

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
from helpers import load_tweets

documents = load_tweets(np.array(filenames)[indeces])

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
processed_docs = [ preprocess(doc) for doc in documents ]
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=14, id2word=dictionary, passes=10, workers=4, alpha=0.01, eta=.91)

for idx, topic in loaded_lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Topic: 0 
Words: 0.008*"lockdown" + 0.007*"reopen" + 0.006*"worker" + 0.006*"extend" + 0.006*"state" + 0.005*"protest" + 0.005*"week" + 0.005*"stay" + 0.004*"home" + 0.004*"covid"
Topic: 1 
Words: 0.020*"case" + 0.017*"death" + 0.010*"report" + 0.009*"covid" + 0.007*"total" + 0.007*"updat" + 0.006*"test" + 0.006*"number" + 0.005*"counti" + 0.005*"confirm"
Topic: 2 
Words: 0.007*"covid" + 0.005*"help" + 0.005*"pandem" + 0.005*"need" + 0.005*"peopl" + 0.004*"work" + 0.004*"care" + 0.004*"health" + 0.004*"mask" + 0.004*"time"
Topic: 3 
Words: 0.005*"mutat" + 0.004*"scientist" + 0.004*"season" + 0.004*"conclud" + 0.0

In [6]:
lda_model.save('trained_lda')

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus_tfidf, dictionary)
vis

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.070634  0.037040       1        1  22.255060
13     0.117656  0.000971       2        1  21.548382
4      0.013344 -0.055014       3        1   8.748424
5      0.007992 -0.013912       4        1   7.250665
9     -0.007600 -0.072063       5        1   6.735568
6      0.007146  0.015006       6        1   5.812305
11    -0.006255 -0.014499       7        1   5.701480
8     -0.006888  0.018016       8        1   4.729131
1     -0.015848  0.021333       9        1   4.053421
0     -0.015925  0.015222      10        1   3.570239
12    -0.038241  0.017466      11        1   2.659904
3     -0.044755  0.005307      12        1   2.345961
2     -0.040615  0.015641      13        1   2.320801
7     -0.040645  0.009485      14        1   2.268659, topic_info=          Term        Freq       Total Category  logprob  loglift
60        test  316.000000  316.000000  Default  30.0000  30.0000
179       case  344.000000  344.000000  Default  29.0000  29.0000
156      trump  270.000000  270.000000  Default  28.0000  28.0000
1794      drug   79.000000   79.000000  Default  27.0000  27.0000
354      china  190.000000  190.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
22    lockdown    5.973697  265.259620  Topic14  -6.0249  -0.0074
360      peopl    4.482670  286.141561  Topic14  -6.3120  -0.3703
292       news    4.226717  252.776494  Topic14  -6.3708  -0.3051
393       poor    3.312951   31.300915  Topic14  -6.6144   1.5402
159      death    3.627611  331.986353  Topic14  -6.5237  -0.7305

[773 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1137      1  0.168195  abus
1137      2  0.126146  abus
1137      3  0.084097  abus
1137      4  0.042049  abus
1137      5  0.042049  abus
...     ...       ...   ...
178       4  0.033648  zone
178       5  0.033648  zone
178       7  0.033648  zone
178       8  0.471079  zone
178      10  0.033648  zone

[5713 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 14, 5, 6, 10, 7, 12, 9, 2, 1, 13, 4, 3, 8])

In [8]:
len(documents)

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


38873